In [1]:
# import keras
import torch

# import numpy as np

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Embedding, LSTM, Dense
# from keras.models import Sequential

In [2]:
# # 1 EU B-ORG
# # 2 rejects O
# # 3 German B-MISC
# # 4 call O
# # 5 to O
# # 6 boycott O
# # 7 British B-MISC
# # 8 lamb O
# # 9 . O

# # The input text, example could be list of sentences
# texts = ['EU rejects German call to boycott British lamb.']

# # The labels corresponding to the input text
# labels = ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']
# # labels = ['B-ORG O B-MISC O O O B-MISC O O']

# # Hyperparameters 
# max_words = 10000 # max number of words to use in the vocabulary
# max_len = 100 # max length of each text (in terms of number of words)
# embedding_dim = 100 # dimension of word embeddings
# lstm_units = 64 # number of units in the LSTM layer
# # num_classes = len(set(labels)) # number of classes

# # Tokenize the texts and create a vocabulary
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(texts)
# sequences = tokenizer.texts_to_sequences(texts)

# # Pad the sequences so they all have the same length
# x = pad_sequences(sequences, maxlen=max_len)

# # Tokenize and convert labels to numerical representation
# label_tokenizer = Tokenizer()
# label_tokenizer.fit_on_texts(labels[0].split())
# label_sequences = label_tokenizer.texts_to_sequences(labels[0].split())
# y = pad_sequences(label_sequences, maxlen=max_len)

# # Convert numerical labels to one-hot encoded labels
# num_classes = len(label_tokenizer.word_index) + 1 # Add 1 for padding
# y = np.eye(num_classes)[y]

# # Ensure x and y have the same number of samples
# assert x.shape[0] == y.shape[0], f"Number of samples in x ({x.shape[0]}) and y ({y.shape[0]}) do not match"

# # Build the model
# model = Sequential()
# model.add(Embedding(max_words, embedding_dim, input_length=max_len))
# model.add(LSTM(lstm_units))
# model.add(Dense(num_classes, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
# model.fit(x, y, batch_size=32, epochs=10)


In [3]:
# sequences

In [4]:
import torch.ao.nn.quantizable as nnqa
rnn = nnqa.LSTM(10, 15, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 15)
c0 = torch.randn(2, 3, 15)
output, (hn, cn) = rnn(input, (h0, c0))
# To get the weights:
print(rnn.layers[0])
print()
print(rnn.layers[0])

_LSTMLayer(
  (layer_fw): _LSTMSingleLayer(
    (cell): QuantizableLSTMCell(
      (igates): Linear(in_features=10, out_features=60, bias=True)
      (hgates): Linear(in_features=15, out_features=60, bias=True)
      (gates): FloatFunctional(
        (activation_post_process): Identity()
      )
      (input_gate): Sigmoid()
      (forget_gate): Sigmoid()
      (cell_gate): Tanh()
      (output_gate): Sigmoid()
      (fgate_cx): FloatFunctional(
        (activation_post_process): Identity()
      )
      (igate_cgate): FloatFunctional(
        (activation_post_process): Identity()
      )
      (fgate_cx_igate_cgate): FloatFunctional(
        (activation_post_process): Identity()
      )
      (ogate_cy): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
  )
)

_LSTMLayer(
  (layer_fw): _LSTMSingleLayer(
    (cell): QuantizableLSTMCell(
      (igates): Linear(in_features=10, out_features=60, bias=True)
      (hgates): Linear(in_features=15, out_features=60, b

# TUTORIAL: [Named Entity Recognition (NER) using Keras LSTM & Spacy](https://zhoubeiqi.medium.com/named-entity-recognition-ner-using-keras-lstm-spacy-da3ea63d24c5) by Beiqi Zhou

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")

from sklearn.model_selection import train_test_split


import tensorflow as tf
print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_80242/57719685.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-12 17:11:23.720464: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow version: 2.15.0
GPU detected: []


In [6]:
data = pd.read_csv("../datasets/ner_practice/ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")
data.index = range(1, len(data) + 1)

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_80242/1133642605.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


In [7]:
data

,Sentence #,Word,POS,Tag
1,Sentence: 1,Thousands,NNS,O
2,Sentence: 1,of,IN,O
3,Sentence: 1,demonstrators,NNS,O
4,Sentence: 1,have,VBP,O
5,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048571,Sentence: 47959,they,PRP,O
1048572,Sentence: 47959,responded,VBD,O
1048573,Sentence: 47959,to,TO,O
1048574,Sentence: 47959,the,DT,O


In [8]:
print("Unique words in corpus:", data['Word'].nunique())
print("Unique tags in corpus:", data['Tag'].nunique())

Unique words in corpus: 35177
Unique tags in corpus: 17


In [39]:
words = list(set(data["Word"].values))
words.append("ENDPAD")


num_words = len(words)
num_words, words

(35178,
 ['2,800',
  'hotspots',
  'benefactor',
  'A-320',
  'funeral',
  'Suleimaniyah',
  'Lyudmila',
  'Whitman',
  'trips',
  'Puli',
  'Soheil',
  'anywhere',
  'Akerfeldt',
  'risen',
  'brother',
  'Bajur',
  'caregivers',
  'larvae',
  'bidding',
  'Leader',
  'AKAEV',
  '196',
  'imprudence',
  'lazy',
  'Herzegovina',
  'heightened',
  'austerity',
  'Desai',
  'logos',
  'Previous',
  'lobbying',
  'primaries',
  'Liban',
  'wife',
  'Tiananmen',
  'Nuriye',
  'Steelers',
  'Timberlake',
  'Nair',
  'differ',
  'post-recession',
  'Nikiforov',
  'after',
  'spark',
  'Reyes',
  'persecutors',
  'Kidwa',
  'nozzles',
  'Lander',
  'quarter',
  'Documents',
  'radioed',
  'astronaut',
  'system',
  'terrorize',
  'Israeli-imposed',
  'endorsement',
  'Thermopylae',
  'Awan',
  'Chi',
  'unrest',
  'rescued',
  'occurs',
  'centrist',
  'tear',
  'immensely',
  'traveling',
  'defenders',
  'Sumaidaie',
  'stages',
  'related',
  'demarcation',
  'Ansar',
  'at-large',
  'shut

In [40]:
word2idx = {w: i + 1 for i, w in enumerate(words)}
word2idx

{'2,800': 1,
 'hotspots': 2,
 'benefactor': 3,
 'A-320': 4,
 'funeral': 5,
 'Suleimaniyah': 6,
 'Lyudmila': 7,
 'Whitman': 8,
 'trips': 9,
 'Puli': 10,
 'Soheil': 11,
 'anywhere': 12,
 'Akerfeldt': 13,
 'risen': 14,
 'brother': 15,
 'Bajur': 16,
 'caregivers': 17,
 'larvae': 18,
 'bidding': 19,
 'Leader': 20,
 'AKAEV': 21,
 '196': 22,
 'imprudence': 23,
 'lazy': 24,
 'Herzegovina': 25,
 'heightened': 26,
 'austerity': 27,
 'Desai': 28,
 'logos': 29,
 'Previous': 30,
 'lobbying': 31,
 'primaries': 32,
 'Liban': 33,
 'wife': 34,
 'Tiananmen': 35,
 'Nuriye': 36,
 'Steelers': 37,
 'Timberlake': 38,
 'Nair': 39,
 'differ': 40,
 'post-recession': 41,
 'Nikiforov': 42,
 'after': 43,
 'spark': 44,
 'Reyes': 45,
 'persecutors': 46,
 'Kidwa': 47,
 'nozzles': 48,
 'Lander': 49,
 'quarter': 50,
 'Documents': 51,
 'radioed': 52,
 'astronaut': 53,
 'system': 54,
 'terrorize': 55,
 'Israeli-imposed': 56,
 'endorsement': 57,
 'Thermopylae': 58,
 'Awan': 59,
 'Chi': 60,
 'unrest': 61,
 'rescued': 62,
 

In [41]:
tags = list(set(data["Tag"].values))
num_tags = len(tags)
num_tags, tags

(17,
 ['I-per',
  'B-org',
  'B-eve',
  'I-nat',
  'I-geo',
  'B-geo',
  'B-tim',
  'I-org',
  'B-art',
  'B-gpe',
  'B-nat',
  'I-eve',
  'B-per',
  'I-tim',
  'O',
  'I-gpe',
  'I-art'])

In [42]:
tag2idx = {t: i for i, t in enumerate(tags)}
tag2idx

{'I-per': 0,
 'B-org': 1,
 'B-eve': 2,
 'I-nat': 3,
 'I-geo': 4,
 'B-geo': 5,
 'B-tim': 6,
 'I-org': 7,
 'B-art': 8,
 'B-gpe': 9,
 'B-nat': 10,
 'I-eve': 11,
 'B-per': 12,
 'I-tim': 13,
 'O': 14,
 'I-gpe': 15,
 'I-art': 16}

In [43]:
import tqdm
def sentence_integrate(data):
  agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                               s["Tag"].values.tolist())]
  return data.groupby('Sentence #').apply(agg_func).tolist()
  # return agg_func(data)
  
sentences=sentence_integrate(data)

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_80242/1614410214.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return data.groupby('Sentence #').apply(agg_func).tolist()


In [44]:
len(sentences)

47959

In [45]:
sentences

[[('Thousands', 'O'),
  ('of', 'O'),
  ('demonstrators', 'O'),
  ('have', 'O'),
  ('marched', 'O'),
  ('through', 'O'),
  ('London', 'B-geo'),
  ('to', 'O'),
  ('protest', 'O'),
  ('the', 'O'),
  ('war', 'O'),
  ('in', 'O'),
  ('Iraq', 'B-geo'),
  ('and', 'O'),
  ('demand', 'O'),
  ('the', 'O'),
  ('withdrawal', 'O'),
  ('of', 'O'),
  ('British', 'B-gpe'),
  ('troops', 'O'),
  ('from', 'O'),
  ('that', 'O'),
  ('country', 'O'),
  ('.', 'O')],
 [('Iranian', 'B-gpe'),
  ('officials', 'O'),
  ('say', 'O'),
  ('they', 'O'),
  ('expect', 'O'),
  ('to', 'O'),
  ('get', 'O'),
  ('access', 'O'),
  ('to', 'O'),
  ('sealed', 'O'),
  ('sensitive', 'O'),
  ('parts', 'O'),
  ('of', 'O'),
  ('the', 'O'),
  ('plant', 'O'),
  ('Wednesday', 'B-tim'),
  (',', 'O'),
  ('after', 'O'),
  ('an', 'O'),
  ('IAEA', 'B-org'),
  ('surveillance', 'O'),
  ('system', 'O'),
  ('begins', 'O'),
  ('functioning', 'O'),
  ('.', 'O')],
 [('Helicopter', 'O'),
  ('gunships', 'O'),
  ('Saturday', 'B-tim'),
  ('pounded', 'O'

In [46]:
sentences[0]

[('Thousands', 'O'),
 ('of', 'O'),
 ('demonstrators', 'O'),
 ('have', 'O'),
 ('marched', 'O'),
 ('through', 'O'),
 ('London', 'B-geo'),
 ('to', 'O'),
 ('protest', 'O'),
 ('the', 'O'),
 ('war', 'O'),
 ('in', 'O'),
 ('Iraq', 'B-geo'),
 ('and', 'O'),
 ('demand', 'O'),
 ('the', 'O'),
 ('withdrawal', 'O'),
 ('of', 'O'),
 ('British', 'B-gpe'),
 ('troops', 'O'),
 ('from', 'O'),
 ('that', 'O'),
 ('country', 'O'),
 ('.', 'O')]

In [47]:
sentences[1]

[('Iranian', 'B-gpe'),
 ('officials', 'O'),
 ('say', 'O'),
 ('they', 'O'),
 ('expect', 'O'),
 ('to', 'O'),
 ('get', 'O'),
 ('access', 'O'),
 ('to', 'O'),
 ('sealed', 'O'),
 ('sensitive', 'O'),
 ('parts', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('plant', 'O'),
 ('Wednesday', 'B-tim'),
 (',', 'O'),
 ('after', 'O'),
 ('an', 'O'),
 ('IAEA', 'B-org'),
 ('surveillance', 'O'),
 ('system', 'O'),
 ('begins', 'O'),
 ('functioning', 'O'),
 ('.', 'O')]

In [48]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max_len = 50
max_len = max(len(sentence) for sentence in sentences)
print(max_len)

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=num_words-1)

104


In [49]:
X

array([[31939, 20173,   443, ..., 35177, 35177, 35177],
       [19216, 29600, 29380, ..., 35177, 35177, 35177],
       [21901,  1701, 33833, ..., 35177, 35177, 35177],
       ...,
       [ 6529, 20524, 10148, ..., 35177, 35177, 35177],
       [23055, 27668,  3893, ..., 35177, 35177, 35177],
       [ 2620, 11126,  6840, ..., 35177, 35177, 35177]], dtype=int32)

In [50]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y

array([[14, 14, 14, ..., 14, 14, 14],
       [ 9, 14, 14, ..., 14, 14, 14],
       [14, 14,  6, ..., 14, 14, 14],
       ...,
       [14,  5, 14, ..., 14, 14, 14],
       [14, 14, 14, ..., 14, 14, 14],
       [14,  1,  7, ..., 14, 14, 14]], dtype=int32)

In [51]:
y

array([[14, 14, 14, ..., 14, 14, 14],
       [ 9, 14, 14, ..., 14, 14, 14],
       [14, 14,  6, ..., 14, 14, 14],
       ...,
       [14,  5, 14, ..., 14, 14, 14],
       [14, 14, 14, ..., 14, 14, 14],
       [14,  1,  7, ..., 14, 14, 14]], dtype=int32)

In [52]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [53]:
len(x_train), len(x_test), len(y_train), len(y_test)

(38367, 9592, 38367, 9592)

In [54]:
x_test

array([[25873,  8839,  3893, ..., 35177, 35177, 35177],
       [ 2620, 18195, 15522, ..., 35177, 35177, 35177],
       [ 2620, 27927, 10225, ..., 35177, 35177, 35177],
       ...,
       [19093, 13165, 20173, ..., 35177, 35177, 35177],
       [ 2620, 11126,   592, ..., 35177, 35177, 35177],
       [33501,  3893, 29451, ..., 35177, 35177, 35177]], dtype=int32)

In [55]:
#build model
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import InputLayer, TimeDistributed, SpatialDropout1D, Bidirectional
from tensorflow import keras

model = keras.Sequential()
model.add(InputLayer((max_len)))
model.add(Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len))
model.add(SpatialDropout1D(0.1))
model.add( Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))

#model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 104, 104)          3658512   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 104, 104)          0         
 alDropout1D)                                                    
                                                                 
 bidirectional_1 (Bidirecti  (None, 104, 200)          164000    
 onal)                                                           
                                                                 
Total params: 3822512 (14.58 MB)
Trainable params: 3822512 (14.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
#compile model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [57]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

chkpt = ModelCheckpoint("model_weights.h5", monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=True, mode='min')


callbacks = [chkpt]

history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test,y_test),
    batch_size=32, 
    epochs=1,
    callbacks=callbacks,
    verbose=1
    
)

1199/1199 [==============================] - ETA: 0s - loss: 0.2183 - accuracy: 0.9687
Epoch 1: val_loss improved from inf to 0.07838, saving model to model_weights.h5
1199/1199 [==============================] - 194s 158ms/step - loss: 0.2183 - accuracy: 0.9687 - val_loss: 0.0784 - val_accuracy: 0.9807


In [58]:
#Apply TensorBoard to check the detailed structure and performance
# %load_ext tensorboard

# %tensorboard --logdir log
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss: {} ".format(results[0]))
print("test accuracy: {} ".format(results[1]))

i = np.random.randint(0, x_test.shape[0])
print("This is sentence:",i)
p = model.predict(np.array([x_test[i]]))
p = np.argmax(p, axis=-1)

print("{:15}{:5}\t {}\n".format("Word", "True", "Pred"))
print("-" *30)
for w, true, pred in zip(x_test[i], y_test[i], p[0]):
    print("{:15}{}\t{}".format(words[w-1], tags[true], tags[pred]))

Evaluate on test data
75/75 [==============================] - 11s 144ms/step - loss: 0.0784 - accuracy: 0.9807
test loss: 0.07837612926959991 
test accuracy: 0.9807000756263733 
This is sentence: 2732
1/1 [==============================] - 1s 733ms/step
Word           True 	 Pred

------------------------------
The            O	O
United         B-geo	B-geo
States         I-geo	I-geo
has            O	O
103            O	O
nuclear        O	O
power          O	O
plants         O	O
in             O	O
31             O	O
states         O	O
.              O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regards        O	O
regar